In [1]:
# import needed library
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math
import numpy as np
import random

# train test split 
from sklearn.model_selection import train_test_split

# Kfold
from sklearn.model_selection import KFold

# regex
import re
 

In [2]:
#load iris dataset
data_iris = load_iris()
iris_X, iris_y = load_iris(return_X_y=True)
feature_iris = data_iris['feature_names']

In [3]:
#transform iris into dataframe
iris_X=pd.DataFrame(iris_X)
iris_y=pd.DataFrame(iris_y)

In [4]:
#create index so be merge
iris_X=iris_X.reset_index()
iris_y=iris_y.reset_index()

In [5]:
iris_X.rename(columns = {0:feature_iris[0],1:feature_iris[1],2:feature_iris[2],3:feature_iris[3]}, inplace = True)

In [6]:
iris_y.rename(columns = {0:'target'}, inplace = True) 

In [7]:
#merge dataset iris
iris=iris_X.merge(iris_y)

In [8]:
#drop index
iris.drop("index",axis=1,inplace=True)

In [9]:
def entropy(parsed_data, target_attribute):
    parsed_value_target = {}
    total_value_target = 0
  
    for i in parsed_data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

            total_value_target += 1
  
    log_result = 0

    for i in parsed_value_target:
        log_result += float(parsed_value_target[i])/total_value_target * math.log((float(parsed_value_target[i])/total_value_target), 2)
  
    return -1 * log_result

In [10]:
# hasn't handle after universal entropy
def information_gain(data, gain_attribute, target_attribute):
    gain_result = 0
    attribute_entropy_result = 0
    parsed_attribute_count = {}
    total_attribute_count = 0
    
    for i in data[gain_attribute]:
        if i is not None:
            if i not in parsed_attribute_count:
                parsed_attribute_count[i] = 1
            else:
                parsed_attribute_count[i] += 1
            
            total_attribute_count += 1
    
    for i in parsed_attribute_count:
        parsed_data = data.loc[data[gain_attribute]==i]
        attribute_entropy_result += float(parsed_attribute_count[i])/total_attribute_count * entropy(parsed_data, target_attribute)    

    gain_result += entropy(data,target_attribute) + (-1 * attribute_entropy_result)
    return gain_result

In [11]:
def split_information(data, gain_attribute):
    res = entropy(data, gain_attribute)
    if(res==0):
        res=0.00000001
    return res

In [12]:
def gain_ratio(data, gain_attribute, target_attribute):
    res = information_gain(data, gain_attribute, target_attribute) / split_information(data, gain_attribute)
    return res

In [13]:
def gain_ratio_continous(data, gain_attribute, target_attribute):
    res = information_gain_continous(data, gain_attribute, target_attribute)[0] / split_information(data, gain_attribute)
    return res,information_gain_continous(data, gain_attribute, target_attribute)[1]

In [14]:
def best_attribute(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': ''
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if information_gain(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = information_gain(data, i, target_attribute)
                    gain_attribute['name'] = i
            else:
                if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                    gain_attribute['name'] = i
                

    return gain_attribute['name']

In [15]:
import math

class Node:
    def __init__(self, attribute=None, label=None, vertex=None):
        self.attribute = attribute
        self.label = label
        self.vertex = vertex
        self.children = {}
        self.most_common_label = None
    
    def set_most_common_label(self, most_common_label):
        self.most_common_label = most_common_label
        
    def get_most_common_label(self):
        return self.most_common_label
        
    def setAttribute(self, attribute):
        self.attribute = attribute

    def setLabel(self, label):
        self.label = label
        
    def setVertex(self, vertex):
        self.vertex = vertex
  
    def addChildren(self, attributeValue, node):
        self.children[attributeValue] = node
    
    def getChildren(self):
        return self.children
    
    def getLabel(self):
        return self.label
    
    def getVertex(self):
        return self.vertex

In [16]:
def get_most_common_label(data, target_attribute):
    parsed_value_target = {}
  
    for i in data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

    most_common = {
        'value': 0,
        'name': ''
    }
    
    for i in parsed_value_target:
        if parsed_value_target[i] > most_common['value']:
            most_common['value'] = parsed_value_target[i]
            most_common['name'] = i
    
    return most_common['name']

In [17]:
def most_common_label_target(data,target_attribute):
    most_comm = None
    occ = 0;
    for i in data[target_attribute].unique():
        if data[data[target_attribute] == i].shape[0] >  occ:
            most_comm = i
            occ = data[data[target_attribute] == i].shape[0]
    return most_comm

In [18]:
def id3(data, target_attribute, is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_ = best_attribute(data,target_attribute,is_IG)
        node.setAttribute(best_attribute_)
        for i in data[best_attribute_].unique():
            node.addChildren(i,id3(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
            node.set_most_common_label(most_common_label_target(data,target_attribute))        
            
    return node

In [19]:
def print_tree(node,depth):
    if node.label is not None: 
        print("    "*(depth+1) +str(node.label))
    else:
        print("    "*depth + "["+ node.attribute +"]")
        for i in node.children:
            print("----"*(depth+1) +str(i))
            print_tree(node.children[i],depth+1)        

In [20]:
def copy_tree(node):
    temp_node = Node()
    if node.label is not None: 
        temp_node.setLabel(node.label)
    else:
        temp_node.setAttribute(node.attribute)
        for i in node.children:
            temp= Node()
            temp= node.children[i]
            temp_node.addChildren(i, temp)
    return temp_node

In [21]:
def check_tree(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        for i in node.children:
            if i==data.loc[index,node.attribute]:
                check_tree(node.children[i],data,index,result, target_attribute)

In [22]:
def pred(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree(model,data[i:i+1],i,result, target_attribute)
        data = {target_attribute:result}
    return pd.DataFrame(data)
    

In [23]:
# hasn't handle after universal entropy
def information_gain_continous(data, gain_attribute, target_attribute):
    gain_result = 0
    save_boundary = -1
    min_entropy = 999999 
    data = data.sort_values(gain_attribute)
    data=data.reset_index().drop('index',axis=1)
    length_data = len(data)
    for i in range(length_data):
        if (i!=length_data-1):
            if (data.loc[i,target_attribute] != data.loc[i+1,target_attribute]):
                temp_boundary = (data.loc[i,gain_attribute] + data.loc[i+1,gain_attribute])/2
                parsed_data_upper = data.loc[data[gain_attribute]>=temp_boundary]
                parsed_data_lower = data.loc[data[gain_attribute]<temp_boundary]
                len_parsed = len(parsed_data_upper)               
                temp_entropy = float(len_parsed)/length_data * entropy(parsed_data_upper, target_attribute) + float((length_data-len_parsed))/length_data * entropy(parsed_data_lower, target_attribute)     
                if temp_entropy < min_entropy:
                    #print(gain_attribute, temp_boundary, temp_entropy)
                    min_entropy = temp_entropy 
                    save_boundary = temp_boundary

    gain_result += entropy(data,target_attribute) + (-1 * min_entropy)
    return gain_result,save_boundary

In [24]:
information_gain_continous(iris, 'petal length (cm)', 'target')

(0.9182958340544894, 2.45)

In [25]:
def best_attribute_c45(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': '',
        'boundary': -99999999
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = information_gain_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if information_gain(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = information_gain(data, i, target_attribute)
                            gain_attribute['name'] = i
            else:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = gain_ratio_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                            gain_attribute['name'] = i

    return gain_attribute['name'],round(gain_attribute['boundary'],2)

In [26]:
best_attribute_c45(iris,'target',False)

/home/ramadhanriandi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


('petal width (cm)', 0.8)

In [27]:
def c45(data, target_attribute,is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
        else:
            node.addChildren('>='+str(bound),c45(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG))
            node.addChildren('<'+str(bound),c45(data.loc[data[best_attribute_]<bound],target_attribute,is_IG))
    return node

In [28]:
def c45_prun(data, target_attribute,is_IG,vertex_=None):
    node = Node(vertex = vertex_)
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
     
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45_prun(data.loc[data[best_attribute_]==i],target_attribute,is_IG,i))
        else:
            node.addChildren('>='+str(bound),c45_prun(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG,'>='+str(bound)))
            node.addChildren('<'+str(bound),c45_prun(data.loc[data[best_attribute_]<bound],target_attribute,is_IG,'<'+str(bound)))
    return node

In [29]:
c45(iris, "target",True)

/home/ramadhanriandi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if sys.path[0] == '':


In [30]:
print_tree(c45(iris, "target",True),0)

/home/ramadhanriandi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if sys.path[0] == '':


[petal length (cm)]
---->=2.45
    [petal width (cm)]
-------->=1.75
        [sepal width (cm)]
------------>=3.15
            [sepal length (cm)]
---------------->=6.05
                    2
----------------<6.05
                    1
------------<3.15
                2
--------<1.75
        [petal length (cm)]
------------>=4.95
            [petal width (cm)]
---------------->=1.55
                [sepal length (cm)]
-------------------->=6.95
                        2
--------------------<6.95
                        1
----------------<1.55
                    2
------------<4.95
            [petal width (cm)]
---------------->=1.65
                    2
----------------<1.65
                    1
----<2.45
        0


In [31]:
def check_tree_c45(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        else:
            for i in node.children:
                #print(i)
                if i[0]=='<':
                    #print(i,2)
                    bound=float(i[1:])
                    if bound>data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                elif i[0]=='>':
                    bound=float(i[2:])
                    #print(i,1)
                    if bound<=data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                else:
                    if i==data.loc[index,node.attribute]:
                        check_tree(node.children[i],data,index,result, target_attribute)
                    

In [32]:
def pred_c45(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree_c45(model,data[i:i+1],i,result, target_attribute)
    data = { target_attribute: data[target_attribute]
            ,'prediction':result}
    return pd.DataFrame(data)
    

In [33]:
p=iris

In [34]:
p=p.drop('target',axis=1)

In [35]:
p=p.reset_index().drop('index',axis=1)

In [36]:
p['sepal width (cm)'] = None

In [37]:
def accuracy(pred,data):
    cnt = 0
    for i in range(len(pred)):
        if pred.loc[i] == data.loc[i]:
            cnt+=1
    return cnt*100/len(pred)

In [38]:
def get_data_validate(data):
    data_column = data.columns
    data_10 = pd.DataFrame(columns=data_column)
    data_90 = pd.DataFrame(columns=data_column)
    #print(data_column)
    count_10 = 0
    count_90 = 0
    
    for i in range(data.shape[0]):
        if(i%10 == 1):
            # Pass the row elements as key value pairs to append() function 
            data_10 = data_10.append(data.loc[[i]] , ignore_index=True)
        else:
            data_90 = data_90.append(data.loc[[i]] , ignore_index=True)
    return data_10, data_90

In [39]:
def prune_tree(node, attribute, vertex):
    if(node.label is not None):
        return node,0
    elif(node.attribute == attribute and node.vertex == vertex):
        node.children = {}
        node.attribute = None
        node.label = node.most_common_label
        
        return node,1
    else:
        for i in node.children:
            a,b = prune_tree(node.children[i], attribute, vertex)
            
        return node,b

In [40]:
def post_pruning(data, node, current_node, children_node, target_attribute):
    if children_node == 0:
        return node
    else:
        for i in current_node.children:
            if current_node.vertex is None: 
                #print(i,1)
                post_pruning(data, node, current_node.children[i], children_node-1, target_attribute)
            else:
                if current_node.attribute is not None:
                    data10, data90 = get_data_validate(data)
                    #print(current_node.attribute,2)
                    save = copy_tree(current_node)
                    temp_node,c = prune_tree(node, current_node.attribute, current_node.vertex)
                    train=data90.drop(target_attribute,axis=1)
                    #print_tree(node,0)
                    temp_node_pred = pred_c45(train, node, target_attribute)
                    #print(temp_node_pred)
                    temp_node_accuracy = accuracy(temp_node_pred[target_attribute], data90[target_attribute])
                    #print(temp_node_accuracy)
                    if temp_node_accuracy == 100:
                        post_pruning(data, temp_node, current_node, children_node, target_attribute)
                    else :
                        
                        node.addChildren(current_node.vertex, save)
                        #print_tree(node,0)


                        post_pruning(data, node, save.children[i], children_node, target_attribute)
        return node

### ANN

In [41]:
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [42]:
def sign(x):
    if (x>0.5):
        return 1
    else:
        return 0

In [43]:
def count_error(target, output):
    return 0.5*(target-output)*(target-output)

In [44]:
class Neuron:
    def __init__(self, out=None, w=None, is_used=None, error=None):
        self.out = out
        self.w = []
        self.is_used = is_used
        self.error = 0
        self.deltaW = []
        
    def set_out(self, out):
        self.out = out
        
    def get_out(self):
        return self.out
    
    def add_deltaW(self, value):
        self.deltaW.append(value)
        
    def get_deltaW(self, index):
        return self.deltaW[index]
    
    def get_arrdW(self):
        return self.deltaW
    
    def set_deltaW(self, index, value):
        self.deltaW[index] = value
        
    def add_w(self, value):
        self.w.append(value)
        
    def set_w(self, index, value):
        self.w[index] = value
    
    def get_w(self, index):
        return self.w[index]
    
    def get_arrW(self):
        return self.w
    
    def set_is_used(self, is_used):
        self.is_used = is_used
        
    def get_is_used(self):
        return self.is_used
    
    def set_error(self, error):
        self.error = error
    
    def get_error(self):
        return self.error

In [45]:
x = [Neuron() for i in range (5)]
y = [x for i in range(5)]

In [46]:
def LenCol(arr):
    return len(arr)
def LenRow(inp,hid,out):
    return max(inp,max(hid)+1,out)
def MakeMatrix(row,col):
    return [[Neuron() for i in range (col)] for j in range(row)]

In [47]:
def printMatrixMLP(matrix):
    for i in range(len(matrix)):
        temp_matrix_out = []
        for j in range(len(matrix[i])):
            temp_matrix_out.append(matrix[i][j].get_arrW())
        print(temp_matrix_out)

In [48]:
def convTarget(output,unique):
    res = []
    for i in range(unique):
        if (i!=output):
            res.append(0)
        else:
            res.append(1)
    return res
    

In [49]:
def resetDeltaWeight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].set_deltaW(k,0)

In [50]:
def initW(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].add_w(np.random.uniform(-1,1)) 
                mat[j][i].add_deltaW(0)

In [51]:
def feedforward(mat,layer):
    for i in range(1,len(layer)):
        if (i!=len(layer)-1):
            mat[layer[i]][i].set_out(1)
        for j in range(layer[i]):
            net = 0;
            for k in range(layer[i-1]):
                #print(k,i-1,j)
                net = net + mat[k][i-1].get_out()*mat[k][i-1].get_w(j)
#             if (i!=len(layer)-1):
            mat[j][i].set_out(sigmoid(net))
#             else:
#                 mat[j][i].set_out(sign(sigmoid(net)))

In [52]:
def get_error_total(matrix, n_output, target_arr):
    output_col = len(matrix[0])-1
    total_error = 0
    for i in range(n_output):
        total_error += 0.5*(target_arr[i] - matrix[i][output_col].get_out())**2
    
    return total_error

In [53]:
def set_output_error(matrix, row, col, target_value):
    output_value = matrix[row][col].get_out()
    #print(output_value, target_value)
#     print(output_value * (1 - output_value) * (target_value - output_value))
    matrix[row][col].set_error(output_value * (1 - output_value) * (target_value - output_value))

In [54]:
def set_hidden_error(matrix, row, col):
    output_value = matrix[row][col].get_out()
    delta_weight_error = 0
    for i in range(len(matrix[row][col].w)):
        delta_weight_error += matrix[row][col].get_w(i) * matrix[i][col+1].get_error()
    matrix[row][col].set_error(output_value * (1 - output_value) * delta_weight_error)

In [55]:
def update_delta_weight(matrix, row, col, idx , learning_rate):
    #print(matrix[i][col+1].get_error(),matrix[row][col].get_out(),row,col)
    #print(row,col+1)
    delta_weight = learning_rate * matrix[row][col].get_error() * matrix[idx][col-1].get_out()
    #print(delta_weight)
    
    current_weight = matrix[idx][col-1].get_deltaW(row)
    matrix[idx][col-1].set_deltaW(row, current_weight + delta_weight)
    #print(idx,col-1,row,current_weight + delta_weight)

In [56]:
def update_weight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                curr_weight = mat[j][i].get_w(k)
                delta_weight = mat[j][i].get_deltaW(k)
                mat[j][i].set_w(k,curr_weight + delta_weight) 

In [57]:
def backpropagation(matrix, layer, learning_rate, target_arr):
    for i in range(len(layer)-1, 0, -1):
        if i == len(layer)-1:
            for j in range(layer[i]):
                set_output_error(matrix, j, i, target_arr[j])
        else:
            for j in range(layer[i]):
                set_hidden_error(matrix, j, i)
        for j in range(layer[i-1]):
            for k in range(layer[i]):
                update_delta_weight(matrix, k, i, j, learning_rate)

In [58]:
hidden_node = [3,4]
def MLP(data,target, hidden_node, epochs, learning_rate):
    output_node = data[target].nunique()
    input_node = len(data.columns)
    hidden_node.insert(0,input_node)
    hidden_node.append(output_node)
    layer_node = hidden_node
    mlp = MakeMatrix(LenRow(input_node,hidden_node,output_node),LenCol(hidden_node))
    initW(mlp,layer_node)
    train = data.drop(target,axis=1)
    batch = len(data)/10;
    for k in range(epochs):
        error = 0
        for i in range(int(batch)):
            for j in range(10):
                for index,col in enumerate(train.columns):
                    mlp[index][0].set_out(data[col][j+i*10])
                mlp[input_node-1][0].set_out(1)
                feedforward(mlp,layer_node)
                backpropagation(mlp, layer_node, learning_rate, convTarget(data[target][j+i*10],output_node))
                error += get_error_total(mlp, len(convTarget(data[target][j+i*10],output_node)), convTarget(data[target][j+i*10],output_node))
            update_weight(mlp,layer_node)
            resetDeltaWeight(mlp,layer_node)
        print(error)
    print()
    print()
    printMatrixMLP(mlp)

    return mlp,layer_node

In [59]:
def predMyMLP(data, target, model, output_layer):
    out = []
    mlp = model[0]
    for i in range(0,data.shape[0],1):
        for index,col in enumerate(data.columns):
            mlp[index][0].set_out(data[col][i])
        mlp[len(data.columns)-1][0].set_out(1)
        feedforward(mlp,model[1])
        output_col = len(mlp[0])-1
        #print(output_col)
        mx=0
        res=-1
        for j in range(output_layer):
            # print(mlp[j][output_col].get_out())
            if mlp[j][output_col].get_out()>mx:
                mx = mlp[j][output_col].get_out()
                res = j
        # print()
        out.append(res)
    data = { target: data[target],
             'prediction':out}
    return pd.DataFrame(data)


# Testing

In [60]:
class_list = iris['target'].unique()

# Create confusion matrix from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
#    Array              class_list
# Output:
#    pandas.DataFrame   confusion_matrix
#
def confusion_matrix(prediction_data, actual_column, prediction_column, class_list):
    actual_data = pd.Categorical(prediction_data[actual_column], categories=class_list)
    prediction_data = pd.Categorical(prediction_data[prediction_column], categories=class_list)
    confusion_matrix = pd.crosstab(actual_data, prediction_data, rownames=['Actual'], colnames=['Predicted'], dropna=False)
    return confusion_matrix

In [61]:
# Get accuracy from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
# Output:
#    float              accuracy
#
def testing_accuracy(prediction_data, actual_column, prediction_column):
    count = 0
    for index, row in prediction_data.iterrows():
        if(row[actual_column] == row[prediction_column]):
            count = count + 1
    return round(count/prediction_data.shape[0],2)

In [125]:
def testing_precision(confusion_matrix):
    conf_matrix = confusion_matrix.to_numpy()
    all_precisions = []
    
    for idx_col in range(len(conf_matrix[0])):
        true_positive = conf_matrix[idx_col][idx_col]
        false_positive = 0
        
        for idx_row in range(len(conf_matrix)):
            if idx_col != idx_row:
                false_positive += conf_matrix[idx_row][idx_col]
                
        temp_precision = true_positive / (true_positive + false_positive)
        
        all_precisions.append(temp_precision)
    
    return sum(all_precisions)/len(all_precisions)

In [128]:
def testing_recall(confusion_matrix):
    conf_matrix = confusion_matrix.to_numpy()
    all_recalls = []
    
    for idx_row in range(len(conf_matrix)):
        true_positive = conf_matrix[idx_row][idx_row]
        false_negative = 0
        
        for idx_col in range(len(conf_matrix[0])):
            if idx_col != idx_row:
                false_negative += conf_matrix[idx_row][idx_col]
                
        temp_recall = true_positive / (true_positive + false_negative)

        all_recalls.append(temp_recall)
    
    return sum(all_recalls)/len(all_recalls)

In [130]:
def testing_f1(precision, recall):
    
    return 2*precision*recall/(precision+recall)

## Train Test Split

In [62]:
# Drop index columns
iris_X.drop('index',axis=1,inplace=True);
iris_y.drop('index',axis=1,inplace=True);

In [63]:
# spliting data
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.1, random_state=42)

In [64]:
# merge data from train test split
# output is train and testing data
def mergeData(X_train, X_test, y_train, y_test):
    X_train = X_train.reset_index()
    y_train = y_train.reset_index()
    X_test = X_test.reset_index()
    y_test = y_test.reset_index()
    trainData = X_train.merge(y_train)
    testData = X_test.merge(y_test)
    trainData.drop("index",axis=1,inplace=True)
    testData.drop("index",axis=1,inplace=True)
    return trainData,testData

In [65]:
trainData,testData = mergeData(X_train, X_test, y_train, y_test)

## DTL

In [99]:
# training model
model = c45(trainData, "target",True)
# predict test
prediction_dtl = pred_c45(testData, model, "target")
print(prediction_dtl)

/home/ramadhanriandi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if sys.path[0] == '':


    target  prediction
0        0           0
1        0           0
2        0           0
3        0           0
4        0           0
5        0           0
6        0           0
7        1           1
8        1           1
9        1           1
10       1           1
11       2           2
12       2           2
13       2           2
14       2           1


In [100]:
print(confusion_matrix(prediction_dtl,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          7  0  0
1          0  4  0
2          0  1  3


In [101]:
testing_accuracy(prediction_dtl,'target','prediction')

0.93

In [113]:
conf_matrix_dtl = confusion_matrix(prediction_dtl,'target','prediction', class_list)

[[7 0 0]
 [0 4 0]
 [0 1 3]]
3.5


In [127]:
print(testing_precision(conf_matrix_dtl))
precision_dtl = testing_precision(conf_matrix_dtl)

0.9333333333333332


In [129]:
print(testing_recall(conf_matrix_dtl))
recall_dtl = testing_recall(conf_matrix_dtl)

0.9166666666666666


In [131]:
print(testing_f1(precision, recall))

0.9249249249249248


## ANN

In [70]:
# training model
hidden_node = [4,4,2]
x = MLP(trainData, 'target', hidden_node,1500,0.1)

45.265949326458745
44.23692759218094
43.775938658377804
43.57295941062258
43.48350826162386
43.44325355817753
43.424280243711955
43.414583649045674
43.408987676031124
43.40523366767439
43.40231432020898
43.399764734426
43.39735910778059
43.39497952107843
43.39255816956601
43.39005126170702
43.38742677993101
43.38465843554409
43.38172246511586
43.37859576984955
43.375254706674994
43.3716741973536
43.36782698364211
43.36368293135964
43.35920832120668
43.35436508009644
43.349109912773244
43.34339329369572
43.337158275281084
43.330339061185605
43.322859282208086
43.3146298970109
43.305546619052414
43.29548674324372
43.28430520849728
43.271829682143284
43.257854384366816
43.24213227857603
43.22436512721868
43.20419073801356
43.18116648250291
43.15474782752477
43.12426013696446
43.08886131151519
43.04749184286452
42.9988074249036
42.941087181102596
42.87210753320679
42.788967326279355
42.68784347039744
42.5636473970875
42.40954057220104
42.21625275802608
41.971133983249565
41.6568751101443
4

12.603482517847073
12.596722069239922
12.590108205785638
12.583634883804196
12.577294407511529
12.571078484553484
12.56497883206542
12.558987524208543
12.553097132836877
12.54730083919743
12.541592335208389
12.535966051386938
12.53041647430711
12.5249398569371
12.519529093681447
12.514189201377938
12.508887343246801
12.503721194182905
12.498325162896782
12.494050785423461
12.485618340007216
12.493824643950106
12.43004649298311
12.539798006823524
12.076625711750959
12.208308930939069
12.384523100184667
12.347023562579137
11.838511587281
11.864322898394454
12.287339423147149
11.830938061144979
11.845241521247313
11.953131518582845
11.883649627522436
12.531999862462683
11.812833930870129
11.8711671142919
12.521326989687367
11.845449413013755
12.372156565202982
12.398694784554959
12.292743199608367
11.93103525502926
11.743684098134244
11.968498467726347
11.954724969193533
11.938781477872395
11.908929688876311
11.76811501633059
11.839983780880637
12.466070156462967
11.981287603258655
11.964

11.579934682980069
11.326992466807177
11.62079116976648
11.232101037872678
11.707686863314889
11.56550265514247
11.60516799837842
11.265647345425895
11.678878958463265
10.708505162476746
11.228776866248095
12.237029497853595
11.170274615197927
11.576639370592977
11.258852325352274
12.422627572165993
11.574865823634106
11.689377925040063
11.700887670328006
11.6754351756762
12.515369494548906
11.53971557368021
11.492065758383228
11.083922982328044
11.045273836699904
11.236370282422042
11.862776104761545
11.191044769908357
11.179134348703558
11.040436344170681
11.191302016660824
11.0500543536543
11.162710567023417
11.025734505961418
11.03233784255556
11.178389861901382
11.078464330601017
11.578526759622083
11.172727780424614
11.007719020176113
11.082437121633559
11.18924607092752
12.114442562849675
11.066258215082394
11.651671833809024
11.016198438070983
11.216643286141903
11.913367157694017
11.20913910078908
12.343552954025611
11.221808820134411
11.722288510113101
11.204989085394594
11.8

10.626329422553363
11.712588070224685
11.579303746976988
10.623831730593976
10.693740755918345
10.76567870570777
10.95563650633121
10.807063111055358
11.244606172480433
10.832058385568944
11.150670702983955
10.684417167514438
10.788928995799653
10.825314447679647
10.77978396183367
11.213924457273889
10.802351938642207
10.782853546637833
11.511729288589315
10.720491488205026
10.613216839447036
10.614597175191756
10.786228548096513
10.782019709018346
10.613538981087128
10.609029389014477
10.58016861745199
10.75208888251178
10.95374233083533
10.863942981298527
10.781555538202754
10.98507687409883
10.751624434651587
10.717534543801577
11.088542453133991
10.784402855935019
10.629622319271196
10.94702318020012
10.842575783978642
10.769855668683894
11.156211513875919
10.68213694047386
10.792259796806984
11.088195355335932
10.696078437181942
10.629614012139603
10.762584969266648
11.824727400737032
11.168228025580449
10.808278442151535
11.114405352749841
10.78749943811294
10.71164620874379
10.6

In [71]:
# predict
prediction_ann = predMyMLP(testData, 'target', x, 3)
print(prediction_ann)

    target  prediction
0        1           1
1        0           0
2        2           2
3        1           1
4        1           1
5        0           0
6        1           1
7        2           2
8        1           2
9        1           1
10       2           2
11       0           0
12       0           0
13       0           0
14       0           0


In [132]:
# confusion matrix
conf_matrix_ann = confusion_matrix(prediction_ann,'target','prediction', class_list)
print(confusion_matrix(prediction_ann,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          7  0  0
1          0  4  0
2          0  0  4


In [133]:
# accurcy
testing_accuracy(prediction_ann,'target','prediction')

1.0

In [ ]:
print(testing_precision(conf_matrix_dtl))
precision_dtl = testing_precision(conf_matrix_dtl)

In [ ]:
print(testing_recall(conf_matrix_dtl))
recall_dtl = testing_recall(conf_matrix_dtl)

In [ ]:
print(testing_f1(precision, recall))

## Cross Validation

## DTL

In [74]:
# make kfold
kf = KFold(n_splits=10,shuffle=True)

In [75]:
# this function will create file to save our DTL model
# our model representation will be likely printed tree

def create_file_DTL(node, depth):
    
    if node.label is not None: 
        file.write(" "*(depth+1) +str(node.label))
        file.write('\n')
    else:
        file.write(" "*depth + "["+ node.attribute +"]")
        file.write('\n')
        for i in node.children:
            file.write("----"*(depth+1) +str(i))
            create_file_DTL(node.children[i],depth+1)        

In [76]:
# load file txt representation of DTL and return an model as root

def load_file_dtl(parent, depth, file):
    line = file.readline().rstrip()

    while line:
        tabs = line.count('----')

        if tabs < depth:
            break
        else :
            node = Node()
            parsed_line = ''
            vertex = ''

            if depth == 0:
                line = line[1:-1]
                parent.setAttribute(line)

            else:
                if (line[-1] == ']'):
                    parsed_line = line.replace('----', '').split('[')
                    vertex = parsed_line[0].rstrip()
                    attribute = parsed_line[1].strip()[:-1]
                    node.setAttribute(attribute)
                    node.setVertex(vertex)
                    
                else:
                    parsed_line = line.replace('----', '').split(' ' * (depth+1))
                    vertex = parsed_line[0].rstrip()
                    label = parsed_line[1].strip()
                    node.setVertex(vertex)
                    node.setLabel(label)

            parent.children[vertex] = node
            
            line = load_file_dtl(node, depth + 1, file)
    
    return line 

In [77]:
# save DTL model into file
file = open("text_dtl.txt", "w")
create_file_DTL(model, 0)
file.close()

In [78]:
# load DTL model from file
node = Node() # node is a parent node
file = open("text_dtl.txt", "r")

#call the function to load DTL model from file, 0 (depth) represent a root node
load_file_dtl(node, 0 , file)
node.children[''].setAttribute(node.attribute)
node = node.children['']
print_tree(node, 0)
file.close()

[petal length (cm)]
---->=2.45
    [petal width (cm)]
-------->=1.8
        [sepal length (cm)]
------------>=5.95
                2
------------<5.95
            [sepal width (cm)]
---------------->=3.1
                    1
----------------<3.1
                    2
--------<1.8
        [petal length (cm)]
------------>=4.95
            [petal width (cm)]
---------------->=1.55
                [sepal length (cm)]
-------------------->=6.95
                        2
--------------------<6.95
                        1
----------------<1.55
                    2
------------<4.95
            [petal width (cm)]
---------------->=1.65
                    2
----------------<1.65
                    1
----<2.45
        0


In [79]:
accuracy_DTL_KF = []
for train_index, test_index in kf.split(iris_X):
    X_train, X_test = iris_X.loc[train_index], iris_X.loc[test_index]
    y_train, y_test = iris_y.loc[train_index], iris_y.loc[test_index]
    trainData,testData =  mergeData(X_train, X_test, y_train, y_test)
    # training model
    model = c45(trainData, "target",True)
    # predict
    prediction_dtl = pred_c45(testData, model, "target")
    # save accuracy result from kfold
    accuracy_DTL_KF.append(testing_accuracy(prediction_dtl,'target','prediction'))

/home/ramadhanriandi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if sys.path[0] == '':


In [80]:
accuracy_DTL_KF

[1.0, 1.0, 1.0, 0.93, 1.0, 0.93, 0.93, 0.93, 0.87, 0.87]

In [81]:
round(np.mean(accuracy_DTL_KF)*100,2)

94.6

## ANN

In [82]:
# this function will create file to save our ANN model
# we save for each line represent a node j on layer i
# for every line we print into file in format; i,j = array_W | array_dW | out | error

def create_file_ANN():
    file = open("text_ann.txt", "w")

    for i in range(0,len(x[1])):
        for j in range(0,x[1][i]):
            file.write(str(i));
            file.write(',')
            file.write(str(j));
            file.write('=')
            file.write(str(x[0][i][j].get_arrW()))
            file.write('|')
            file.write(str(x[0][i][j].get_arrdW()))
            file.write('|')
            file.write(str(x[0][i][j].get_out()))
            file.write('|')
            file.write(str(x[0][i][j].get_error()))
            file.write('\n');

    file.close()

In [83]:
# load file txt that contain model representation of ANN

def load_file_ann(file):
    neurons = file.read().split("\n")[:-1]

    rows = []
    columns = []

    curr_row = 0

    for neuron in neurons:
        neuron_pos = neuron.split('=')[0].split(',')
        if int(neuron_pos[0]) != curr_row:
            rows.append(columns)
            columns = []
            curr_row += 1

        new_neuron = Neuron()
        neuron_elements = neuron.split('=')[1].rstrip().split('|')
        weights = neuron_elements[0]
        delta_weights = neuron_elements[1]
        out = neuron_elements[2]
        err = neuron_elements[3]

        if len(weights) > 2:
            parsed_weights = weights[1:-1].split(',')
            for weight in parsed_weights:
                new_neuron.add_w(float(weight.strip()))
        
        if len(delta_weights) > 2:
            parsed_delta_weights = delta_weights[1:-1].split(',')
            for delta_weight in parsed_delta_weights:
                new_neuron.add_deltaW(float(delta_weight.strip()))

        new_neuron.set_out(float(out))
        new_neuron.set_error(float(err))

        columns.append(new_neuron)

    max_col_len = max([len(row) for row in rows])
    for row in rows:
        if len(row) < max_col_len:
            for i in range(max_col_len-len(row)):
                row.append(Neuron())

    return rows

In [84]:
# save ANN model into file
create_file_ANN()

In [85]:
# load ANN model from file
file = open("text_ann.txt", "r")
rows = load_file_ann(file)
printMatrixMLP(rows)
file.close()

[[1.448653375638878, -5.291280212373386, 0.5079757526901176, -0.8162735483808092], [1.38495749284236, -1.5789529199896688, 2.783478347742112, -0.8983176547332128], [3.8338090096949053, -1.4412137895244954], [3.278139751399117, -2.005379804979942, -21.830552462960714], []]
[[-0.5506012607893529, -9.10992006779075, 0.3635544499483121, -1.6181880344919142], [-1.2983263791611153, 2.0057944339264635, -7.795180424971367, -0.16099225626920535], [-6.102684152523247, 5.505139455227125], [-21.71260871897396, -1.8345561935114714, 3.0452905414809037], []]
[[0.16206588085949805, 10.490541111097876, 1.0290758684638712, 2.983105555852818], [0.2696719434824747, -0.6714829006705733, 3.643777901585897, -0.4006487729130446], [2.9612119646244826, -7.0633518771402946], [], []]
[[0.748101995125572, 11.28833418901994, -0.232527973337095, 2.047970954704169], [-3.5469540406546116, 4.8723793663737975, -0.4586613901790655, -0.4221093838952048], [], [], []]


In [86]:
accuracy_MLP_KF = []
for train_index, test_index in kf.split(iris_X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = iris_X.loc[train_index], iris_X.loc[test_index]
    y_train, y_test = iris_y.loc[train_index], iris_y.loc[test_index]
    trainData,testData =  mergeData(X_train, X_test, y_train, y_test)
    hidden_node = [4,4,2]
    # train
    model = MLP(trainData, 'target', hidden_node,1500,0.1)
    # predict
    prediction_ann = predMyMLP(testData, 'target', x, 3)
    # save accuracy result from kfold
    accuracy_MLP_KF.append(testing_accuracy(prediction_ann,'target','prediction'))

54.05744134430608
47.99494696594519
45.79150484720947
44.914859474134296
44.59414266270171
44.48071325030309
44.433261850024024
44.403024694592176
44.37545058464968
44.34716186970522
44.31804097159364
44.28871029419565
44.259748416276516
44.23146049965719
44.20388317566036
44.17690442838378
44.15038855452453
44.124239720154755
44.09840672289328
44.072861590473906
44.04757679425348
44.022510140972464
43.99759727406834
43.9727489838411
43.94785071455056
43.9227625470147
43.897318697019614
43.87132604416052
43.844561445734904
43.81676767706982
43.78764783171347
43.75685794401198
43.72399747411623
43.6885971186584
43.650103166098205
43.607857282235805
43.56107015784918
43.50878683464355
43.449840687783336
43.38279189261459
43.305844594175895
43.2167346868145
43.11257667718156
42.98965279670557
42.84311892910139
42.66658708768932
42.451516360129126
42.18628623039613
41.854691125126244
41.433259586645754
40.88608568267558
40.15629117068846
39.16968477309593
37.895313903921505
36.374259224963

13.998971419404189
13.990273757197079
13.981550934051155
13.972803521218527
13.964032149643907
13.955237511915975
13.94642036414246
13.937581527730304
13.928721891051941
13.91984241097715
13.910944114249494
13.902028098685461
13.893095534173986
13.884147663453248
13.875185802642223
13.866211341503364
13.857225743414244
13.848230545025837
13.839227355585924
13.830217855908067
13.821203796966994
13.812186998104366
13.803169344829906
13.79415278620663
13.785139331811132
13.776131048263444
13.767130055324852
13.758138521565751
13.74915865960987
13.740192720966153
13.731242990463413
13.722311780308562
13.713401423792968
13.70451426867753
13.695652670290313
13.686818984376291
13.67801555974184
13.669244730741337
13.66050880965564
13.651810079015698
13.643150783926254
13.634533124446088
13.625959248082179
13.617431242455025
13.608951128191269
13.600520852098429
13.592142280673938
13.58381719399772
13.57554728005317
13.567334129517207
13.559179231055206
13.551083967150317
13.543049610491572
13

11.928874093210025
11.92636828433756
11.923862355803209
11.921356232245595
11.918849837810281
11.916343096127756
11.91383593029103
11.911328262833216
11.908820015704537
11.906311110249264
11.903801467182458
11.901291006566408
11.89877964778673
11.896267309528458
11.893753909751856
11.89123936566774
11.888723593713184
11.886206509526373
11.88368802792181
11.881168062865003
11.8786465274472
11.876123333859898
11.873598393369221
11.871071616290264
11.868542911961208
11.866012188717486
11.863479353865818
11.86094431365829
11.858406973266359
11.855867236754817
11.853325007055874
11.850780185943426
11.848232674007166
11.84568237062682
11.843129173946892
11.840572980851283
11.838013686938167
11.835451186495208
11.832885372475145
11.830316136471538
11.827743368695097
11.825166957950337
11.82258679161283
11.820002755606808
11.817414734383647
11.814822610900713
11.812226266601115
11.809625581394208
11.807020433636824
11.804410700115534
11.801796256029837
11.799176974976376
11.796552728934342
11.

11.231394526396814
11.220233886950368
11.219684286911354
11.213453308386095
11.209523391972532
11.20461439418242
11.199750576177449
11.194687330887419
11.189400533461425
11.183852292256272
11.177979065547188
11.171689522977521
11.164846220938767
11.15722794649553
11.148449106001477
11.137765614671482
11.123539946022925
11.101560060619573
11.061334115081424
11.056506608548125
11.166208285149164
11.08487032086369
11.07655839173817
11.137962032590398
11.077638122661622
11.106846229767608
11.094470542098623
11.101003559732225
11.09221689145661
11.093579326269408
11.089378543128815
11.087602215667191
11.085083732296994
11.082668939570386
11.0803067181651
11.077949893365918
11.07560784094272
11.07329139383121
11.07101147385186
11.068791593875734
11.066691784129926
11.064880448474636
11.063905548786362
11.065952151650734
11.08183201020678
11.147710632303564
11.323862028694606
11.244936396576533
11.293659695168836
11.254578911564703
11.27229729911647
11.257978394180967
11.260457241969881
11.25

17.235520451788556
17.199103049424277
17.162888589450883
17.126877993443767
17.091070262656466
17.05546315714325
17.020053715080614
16.984838635722515
16.9498145510688
16.91497820987135
16.88032659465699
16.845856989003245
16.81156700892688
16.777454609213713
16.743518072944337
16.709755990364975
16.676167231579484
16.64275091623848
16.6095063824129
16.57643315609915
16.54353092226107
16.510799497924783
16.47823880756884
16.44584886086697
16.41362973271972
16.381581545438074
16.349704452902678
16.31799862650681
16.286464242691835
16.25510147189565
16.22391046875173
16.192891363398527
16.162044253780273
16.131369198843153
16.100866212550876
16.07053525866198
16.040376246227428
16.010389025779777
15.980573386195712
15.950929052221506
15.921455682656111
15.89215286919011
15.863020135899978
15.834056939397685
15.805262669634196
15.776636651354673
15.748178146201209
15.719886355457069
15.691760423425475
15.66379944143515
15.636002452464433
15.608368456376972
15.580896415763387
15.5535852623

...

20.981807188213576
20.981452568351777
20.981083926807663
20.980699740064956
20.980298289215128
20.97987763330775
20.97943557977028
20.978969652093447
20.97847705533714
20.97795464056982
20.97739887019709
20.976805787363617
20.976170994348706
20.975489647232443
20.9747564771494
20.973965852092352
20.97311189710417
20.97218869389904
20.97119058177989
20.970112577441622
20.96895091826618
20.967703708488656
20.96637160892789
20.96495846351906
20.963471713683806
20.961922437841462
20.960324892466925
20.958695532241197
20.95705162795491
20.955409728485265
20.953784267581838
20.952186568717313
20.950624374840693
20.949101883982145
20.947620164321073
20.94617778020947
20.944771475591995
20.943396806881594
20.942048667862032
20.94072168924672
20.939410520520955
20.93811001359159
20.936815330838314
20.935521998427138
20.93422592204462
20.93292337818184
20.931610990512848
20.93028569803469
20.928944719460862
20.92758551678596
20.92620575982955
20.924803292808303
20.923376103474776
20.921922295028

12.321818769122434
13.933217187547301
13.144630739203482
14.24476076631625
12.489356640711627
13.781511211891113
11.643179385439428
12.636796573238108
14.006751593048389
12.520755476153761
13.840205165988774
12.996817957293448
13.706245786579968
11.629877045327287
12.504809688263204
13.787916048192137
13.078490123699547
14.105497848617999
13.960965295508025
13.89972845814079
13.916019789787951
13.90660404222456
13.8875927668025
13.870050632705626
13.851929768554559
13.832536110016333
13.80238320594447
13.611652247233513
13.492863557875982
13.65326553333789
13.60354258375004
12.095385442970663
13.764505042693392
13.667111358441634
13.387083877747154
12.31373929430013
13.513161701669416
12.18580305081296
13.595084785629187
13.508643055970621
13.669579127476515
12.025359940105881
13.701829422738955
13.369232893466748
13.384323977930316
11.961647974408596
12.219283656253829
13.451398160856526
13.585639538817409
13.267014900979317
13.230037979358467
12.575516588865488
13.605278761194267
11.

20.135017507414062
20.102152372650412
20.06908016623468
20.035796376962075
20.00229745580839
19.968580923137978
19.93464547637507
19.900491096807404
19.86611915398774
19.83153250589559
19.796735592615292
19.761734520779015
19.726537135429112
19.69115307530233
19.6555938068797
19.619872631952887
19.584004663030925
19.548006760770473
19.511897427899562
19.47569665495447
19.439425714681843
19.4031069042367
19.366763237311833
19.3304180919301
19.29409482354531
19.257816356934754
19.221604773629846
19.185480913804984
19.149464012177525
19.11357138628433
19.07781819246171
19.04221726023085
19.006779010102008
18.971511453768123
18.936420270011766
18.90150894506142
18.86677896305595
18.832230030887228
18.7978603219081
18.763666724509974
18.729645083973033
18.695790428818665
18.662097175749793
18.62855930984917
18.595170538836285
18.561924421786777
18.528814473808787
18.495834248815303
18.46297740282882
18.430237740304396
18.397609245850813
18.365086103539266
18.332662705768097
18.3003336534332

14.165195920700356
14.161784673403671
14.15842398829821
14.15511203905996
14.151847108303905
14.14862758094079
14.145451938186834
14.142318752138456
14.139226680833941
14.13617446373611
14.13316091757826
14.130184932523774
14.127245468596783
14.124341552346978
14.12147227371741
14.118636783087938
14.11583428847176
14.11306405284527
14.110325391594818
14.107617670066784
14.104940301208922
14.102292743294159
14.099674497718352
14.097085106866407
14.094524152041528
14.091991251453985
14.089486058266854
14.087008258696997
14.084557570170347
14.082133739531475
14.079736541307865
14.077365776030135
14.07502126861003
14.072702866778243
14.070410439584982
14.06814387596656
14.065903083381613
14.063687986521233
14.061498526097532
14.05933465771583
14.057196350835888
14.055083587828074
14.05299636313101
14.050934682517262
14.04889856247464
14.046888029710344
14.044903120786387
14.04294388189455
14.041010368779657
14.039102646820407
14.03722079127709
14.03536488771579
14.033535032618795
14.031731

12.895633886866891
12.89346449913124
12.891299686491386
12.889139449803139
12.886983790146495
12.8848327088298
12.882686207393911
12.880544287616843
12.87840695151818
12.876274201363861
12.874146039671077
12.872022469213345
12.869903493025658
12.867789114409922
12.8656793369406
12.863574164470256
12.861473601135772
12.859377651364262
12.857286319879591
12.855199611708922
12.853117532189373
12.851040086975166
12.848967282044862
12.846899123708816
12.844835618617036
12.842776773767065
12.840722596512345
12.838673094570934
12.836628276034086
12.834588149375799
12.832552723461992
12.830522007560637
12.828496011351694
12.826474744937912
12.82445821885549
12.822446444085582
12.820439432065976
12.818437194703112
12.816439744384862
12.81444709399332
12.812459256918501
12.810476247072286
12.808498078902861
12.80652476740989
12.804556328159968
12.802592777303088
12.800634131589124
12.798680408385493
12.796731625695122
12.794787802175483
12.792848957157744
12.790915110667395
12.788986283445157
12

In [87]:
accuracy_MLP_KF

[1.0, 0.87, 1.0, 0.93, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [88]:
round(np.mean(accuracy_MLP_KF)*100,2)

98.0

# Generate New Instance

In [89]:
# Generate n instance like iris dataset
# return new n instance as new dataframe
def generateIris(n):
    a = []
    b = []
    c = []
    d = []
    e = []
    for i in range(n):
        a.append(random.uniform(4.5, 7.5))
        b.append(random.uniform(2, 4.4))
        c.append(random.uniform(1, 6.9))
        d.append(random.uniform(0.1, 2.5))
        e.append(1)
    iris = {'sepal length (cm)': a,
            'sepal width (cm)': b,
            'petal length (cm)':c,
            'petal width (cm)':d,
            'target': e
            }
    return pd.DataFrame(iris, columns = ['sepal length (cm)', 'sepal width (cm)','petal length (cm)','petal width (cm)','target'])

In [90]:
# Make 100 instance
new_ins = generateIris(100)

In [91]:
# save model
model = c45(iris, "target",True)
file = open("text_dtl.txt", "w")
create_file_DTL(model, 0)
file.close()

/home/ramadhanriandi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if sys.path[0] == '':


In [92]:
# load model
node = Node()
file = open("text_dtl.txt", "r")
load_file_dtl(node, 0 , file)
node.children[''].setAttribute(node.attribute)
node = node.children['']
file.close()

In [93]:
# prediction model
prediction_dtl = pred_c45(new_ins, node, "target")

In [94]:
new_ins['target']=prediction_dtl['prediction']

In [95]:
new_ins['target'].value_counts()

2    44
1    35
0    21
Name: target, dtype: int64